In [95]:
!dotnet run --theta1 0.001 --theta2 -0.001 --theta3 0.001 --n-samples 10000000

=====/Users/guen/repos/microsoft/qdk-python/examples/vqe/StatePreparation.qs(14,13): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/StatePreparation.qs(18,17): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/StatePreparation.qs(24,21): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/Utils.qs(32,17): warning QS3306: Deprecated syntax. Parentheses here are no l

=Program.qs(95,25): warning CS0219: The variable 'j' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
Program.qs(55,17): warning CS0219: The variable 'energyOffset' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
StatePreparation.qs(85,17): warning CS0219: The variable 'trotterOrder' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
StatePreparation.qs(86,17): warning CS0219: The variable 'maxTime' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
==-1.116257281695194
=

In [102]:
import subprocess
import json

def run_vqe(theta1, theta2, theta3, term_type=0, n_ham=0, n_op=0, shots=1, target="ionq.simulator"):
#     cmd = f"az quantum execute --target-id {target} -- --theta1 {theta1} --theta2 {theta2} --theta3 {theta3} --term-type {term_type} --n-ham {n_ham} --n-op {n_op} --shots {shots}"
    cmd = f"dotnet run --theta1 {theta1} --theta2 {theta2} --theta3 {theta3} --n-samples {shots}"
    run_cmd = subprocess.run([cmd], check=True, shell=True, capture_output=True)
    result = run_cmd.stdout
    result = json.loads(run_cmd.stdout.lstrip(b"."))
    return result, run_cmd

In [100]:
run_vqe(theta1=0.001, theta2=-0.001, theta3=0.001, shots=10000000)

(-1.1162117065141186,
 CompletedProcess(args=['dotnet run --theta1 0.001 --theta2 -0.001 --theta3 0.001 --n-samples 10000000'], returncode=0, stdout=b'-1.1162117065141186\n', stderr=b''))

In [103]:
from scipy.optimize import minimize

def run_program(var_params, num_samples) -> float:
    # run parameterized quantum program for VQE algorithm
    theta1, theta2, theta3 = var_params
    print(var_params)
    try:
        result, _ = run_vqe(theta1, theta2, theta3, shots=num_samples)
    except Exception:
        result, _ = run_vqe(theta1, theta2, theta3, shots=num_samples)
    print(result)
    return result
 
def VQE(initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
 
    opt_result = minimize(run_program,
                          initial_var_params,
                          args=(num_samples,),
                          method="COBYLA",
                          tol=0.000001,
                          options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05})
 
    return opt_result

In [104]:
%%time
VQE([0.001, -0.001, 0.001], 10000000)

[ 0.001 -0.001  0.001]
-1.1162904403204759
[ 0.051 -0.001  0.001]
-1.1147831214916257
[0.001 0.049 0.001]
-1.1147708037854378
[ 0.001 -0.001  0.051]
-1.0941606170468448
[-0.00238981 -0.00441751 -0.04876776]
-1.1306541113504678
[-0.00762944 -0.00969996 -0.09821108]
-1.1369010682520906
[-0.04165423 -0.04376517 -0.11169718]
-1.1347810779260992
[-0.0253149   0.00796991 -0.09822473]
-1.1365687974739198
[ 0.01334517  0.00546506 -0.14099061]
-1.135870201782455
[ 0.00654065 -0.00389624 -0.07844936]
-1.135309774158713
[-0.0146762  -0.01941685 -0.10170038]
-1.1365152745808964
[-0.00267064 -0.00623116 -0.10914853]
-1.1372413120383027
[ 0.00830927 -0.00100108 -0.11203606]
-1.137412806747494
[ 0.00482351  0.01037618 -0.11586456]
-1.137224723756669
[ 0.01380948 -0.00749392 -0.12119252]
-1.137091349777025
[ 0.01126165 -0.00024664 -0.10657926]
-1.1370100797702345
[ 0.00545066 -0.00226362 -0.11203818]
-1.1372083427706685
[ 0.00893796 -0.00242431 -0.11217944]
-1.1371916401232571
[ 0.00829133  0.00169847

     fun: -1.1372908556920143
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 44
  status: 1
 success: True
       x: array([ 0.00756478, -0.00122729, -0.11203378])

In [109]:
import subprocess
import json

def run_vqe(theta1, theta2, theta3, term_type=0, n_ham=0, n_op=0, shots=1, target="ionq.simulator"):
    cmd = f"az quantum execute --target-id {target} -- --theta1 {theta1} --theta2 {theta2} --theta3 {theta3} --term-type {term_type} --n-ham {n_ham} --n-op {n_op} --shots {shots}"
    run_cmd = subprocess.run([cmd], check=True, shell=True, capture_output=True)
    result = run_cmd.stdout
    result = json.loads(run_cmd.stdout.lstrip(b"."))
    return result, run_cmd

In [135]:
!az quantum execute --target-id ionq.simulator -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0

Command group 'quantum' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
Failed to compile program.


In [136]:
!dotnet run --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0

=====/Users/guen/repos/microsoft/qdk-python/examples/vqe/Utils.qs(32,17): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/Utils.qs(69,17): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/Utils.qs(82,17): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be supported in the future. [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
/Users/guen/repos/microsoft/qdk-python/examples/vqe/Utils.qs(86,21): warning QS3306: Deprecated syntax. Parentheses here are no longer required and will not be su

=StatePreparation.qs(85,17): warning CS0219: The variable 'trotterOrder' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
StatePreparation.qs(86,17): warning CS0219: The variable 'maxTime' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
Program.qs(46,17): warning CS0219: The variable 'energyOffset' is assigned but its value is never used [/Users/guen/repos/microsoft/qdk-python/examples/vqe/VariationalQuantumEigensolver.csproj]
==One
=

In [113]:
run_vqe(theta1=0.001, theta2=-0.001, theta3=0.001, shots=1)

CalledProcessError: Command '['az quantum execute --target-id ionq.simulator -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0 --shots 1']' returned non-zero exit status 1.

In [110]:
hamiltonian = (
    [
        ([0], [0.17120128499999998]),
        ([1], [0.17120128499999998]),
        ([2], [-0.222796536]),
        ([3], [-0.222796536])
    ],
    [
        ([0, 1], [0.1686232915]),
        ([0, 2], [0.12054614575]),
        ([0, 3], [0.16586802525]),
        ([1, 2], [0.16586802525]),
        ([1, 3], [0.12054614575]),
        ([2, 3], [0.1743495025])
    ],
    [],
    [
        ([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])
    ]
)

In [111]:
target = "ionq.simulator"
results = []
shots = 100
def run_vqe_all(theta1, theta2, theta3, target, shots):
    results = []
    for term_type in [0,1,3]:
        if term_type == 0:
            n_hams = range(4)
        elif term_type == 1:
            n_hams = range(6)
        elif term_type == 3:
            n_hams = [0]
        for n_ham in n_hams:
            n_ops = [0] if term_type != 3 else range(4)
            for n_op in n_ops:
                results.append(run_vqe(theta1, theta2, theta3, term_type, n_ham, 0, shots, target))
    return results

In [112]:
def estimate_energy(hamiltonian, results, offset=-0.09883444600000002):
    termExpectation = [a["Histogram"][1] for a, _ in results]
    coeff = [__c for t in hamiltonian for _t, _c in t for __c in _c]
    return sum([(2. * _t - 1.) * _c for _t, _c in zip(termExpectation, coeff)]) + offset;

In [44]:
%%time
results = run_vqe_all(theta1=0.001, theta2=-0.001, theta3=0.001, target="ionq.simulator", shots=100)
estimate_energy(hamiltonian, results)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
from scipy.optimize import minimize

def run_program(var_params, num_samples) -> float:
    # run parameterized quantum program for VQE algorithm
    theta1, theta2, theta3 = var_params
    print(var_params)
    results = run_vqe_all(theta1, theta2, theta3, target="ionq.simulator", shots=num_samples)
    result = estimate_energy(hamiltonian, results)
    print(result)
    return result
 
def VQE(initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
 
    opt_result = minimize(run_program,
                          initial_var_params,
                          args=(num_samples,),
                          method="COBYLA",
                          tol=0.000001,
                          options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05})
 
    return opt_result

In [23]:
%%time
VQE([0.001, -0.001, 0.001], 100)

[ 0.001 -0.001  0.001]
-1.017850997148085
[ 0.051 -0.001  0.001]
-1.0177284071577128
[0.001 0.049 0.001]
-1.0177378414205331
[ 0.001 -0.001  0.051]
-1.0175949100166082
[-0.01905492 -0.01951154 -0.04089419]
-1.0176506599575217
[-0.00902746 -0.01025577 -0.01994709]


CalledProcessError: Command '['az quantum execute --target-id ionq.simulator -- --theta1 -0.009027462038001206 --theta2 -0.010255770058909593 --theta3 -0.019947093490339837 --term-type 3 --n-ham 0 --n-op 0 --shots 100']' returned non-zero exit status 1.

In [14]:
!az quantum execute --target-id ionq.simulator -- --theta1 -0.02 --theta2 -0.02 --theta3 -0.04 --term-type 0 --n-ham 0 --n-op 0 --shots 100

Command group 'quantum' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
....
{
  "Histogram": [
    "0",
    0.0001254914085888,
    "1",
    0.9998745085913923
  ]
}


In [159]:
target = "ionq.simulator"
results = []
shots = 100
for term_type in [0,1,3]:
    if term_type == 0:
        n_hams = range(4)
    elif term_type == 1:
        n_hams = range(6)
    elif term_type == 3:
        n_hams = [0]
    for n_ham in n_hams:
        n_ops = [0] if term_type != 3 else range(4)
        for n_op in n_ops:
            results.append(run_vqe(term_type, n_ham, 0, shots, target))

TypeError: run_vqe() missing 2 required positional arguments: 'n_op' and 'shots'

In [134]:
estimate_energy(hamiltonian, results)

-0.5356664127610464

In [96]:
estimate_energy(hamiltonian, results)

-1.017848070739604

In [60]:
[a for a, _ in results]

[{'Histogram': ['0', 2.4999997919999996e-07, '1', 0.9999997500000226]},
 {'Histogram': ['0', 2.2499968124999997e-06, '1', 0.9999977500031892]},
 {'Histogram': ['0', 0.9999997500000226, '1', 2.4999997919999996e-07]},
 {'Histogram': ['0', 0.9999977500031892, '1', 2.2499968124999997e-06]},
 {'Histogram': ['0', 0.9999975000043351, '1', 2.4999956667e-06]},
 {'Histogram': ['0', 4.999998333999999e-07, '1', 0.9999995000001684]},
 {'Histogram': ['0', 2.4999956667e-06, '1', 0.9999975000043351]},
 {'Histogram': ['0', 2.4999956667e-06, '1', 0.9999975000043351]},
 {'Histogram': ['0', 4.4999835e-06, '1', 0.9999955000165016]},
 {'Histogram': ['0', 0.9999975000043351, '1', 2.4999956667e-06]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]},
 {'Histogram': ['0', 1.0000000000000018]}]

In [ ]:
(2. * termExpectation - 1.) * coeff

In [27]:
import qsharp.azure

In [28]:
resource_id = "/subscriptions/9006b025-1a81-4f7c-9111-582e121509ac/resourcegroups/azure-quantum-guenp/providers/Microsoft.Quantum/Workspaces/azure-quantum-01"
qsharp.azure.connect(resourceId=resource_id)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RLMVK4XTS to authenticate.

Connected to Azure Quantum workspace azure-quantum-01 in location westus.


[{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 102},
 {'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-1.0', 'current_availability': 'Degraded', 'average_queue_time': 0},
 {'id': 'honeywell.hqs-lt-1.0-apival', 'current_availability': 'Available', 'average_queue_time': 0}]

In [29]:
qsharp.azure.target("ionq.simulator")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RXNAYXYWA to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}

In [12]:
result1 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RR6EFCWTY to authenticate.

Submitting Microsoft.Quantum.Samples.Chemistry.SimpleVQE.Test to target ionq.simulator...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: 50f3389d-38fa-4303-aa3b-0784ed5ff619
Waiting up to 30 seconds for Azure Quantum job to complete...
[5:51:08 PM] Current job status: Succeeded


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RCXY9GVFK to authenticate.

In [19]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=0, nHam=0, nOp=0)

Total ham terms: 4
Total measurement operations: 1
0


1

In [30]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=1, nHam=0, nOp=0)

0

In [32]:
Test.simulate(theta1=0.001, theta2=-0.001, theta3=0.001, termType=3, nHam=0, nOp=0)

1

In [22]:
qsharp.azure.target("ionq.qpu")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R667Y5V5A to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 33}

In [23]:
result1 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R7RSJMRRA to authenticate.

Submitting Test to target ionq.qpu...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: f4464075-3854-4b47-adaa-e95f06a8a474
Waiting up to 30 seconds for Azure Quantum job to complete...
[5:15:31 PM] Current job status: Waiting
[5:15:37 PM] Current job status: Waiting
[5:15:42 PM] Current job status: Waiting
[5:15:47 PM] Current job status: Waiting
[5:15:52 PM] Current job status: Waiting


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FG55TVW47 to authenticate.

In [11]:
print(f'Azure Quantum service execution result: {result1}')

Azure Quantum service execution result: {'0': 2.4999997919999996e-07, '1': 0.9999997500000226}


In [12]:
qsharp.azure.target("ionq.qpu")

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RTQH6AYM9 to authenticate.

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 50}

In [13]:
result2 = qsharp.azure.execute(
    Test,
    theta1=0.001,
    theta2=-0.001,
    theta3=0.001,
    termType=0,
    shots=100,
    nHam=0,
    nOp=0,
    jobName="Simulate Hydrogen using VQE"
)
print(f'Azure Quantum service execution result: {result2}')

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RGBHQD3KZ to authenticate.

Submitting Test to target ionq.qpu...
Job successfully submitted for 100 shots.
   Job name: Simulate Hydrogen using VQE
   Job ID: cb123e27-7467-4844-a5bb-3fbbbe73a4e7
Waiting up to 30 seconds for Azure Quantum job to complete...
[4:58:14 PM] Current job status: Waiting
[4:58:19 PM] Current job status: Waiting
[4:58:24 PM] Current job status: Succeeded


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RR5PHG6QF to authenticate.

Azure Quantum service execution result: {'0': 0.01, '1': 0.99}


In [13]:
import subprocess

In [ ]:
subprocess.run